In [1]:
### IMPORTS
from simulate import run_simulation

### Run Single Simulation

In [ ]:
# Number of unique monomers
N_MONs = 4

# Molar ratios of each monomer
# ex. if N_MONs = 4: [0.5, 0.25, 0.20, 0.05]
MRs = [50, 25, 20, 5]

# What model of copolymerization to use? (1) mayolewis (terminal) or (2) penultimate
# NOTE FOR NOW ONLY MAYO-LEWIS IMPLEMENTED.
#model = "mayolewis"

# reactivity ratios of monomers, ex. for 4 monomers:
#[r12, r13, r14]
#[r21, r23, r24]
#[r31, r32, r34]
#[r41, r42, r43]
RRs = ([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]])

# % conversion targetted (0-1), i.e. how much of the monomer pool do you want to use
conv = 0.5

# average degree of polymerization (chain length) you are targetting at YOUR conversion, NOT at 100%.
avgDP = 100

# number of polymer chains to simulate
N_CHAINs = 10000

# Chain transfer % (0-1)
# TODO: replace this with direct PDI control (not sure how)
# NOTE TO SELF: for now just fix CTP such that PDI ~ 1.2, as thats roughly the control our RAFT synthesis gives.
CTP = 0.05

# cutoff DP of chains considered as polymers not oligomers that get "purified" out
# set to 0 if you don't want to do any filtration
PRUNE_OLIGOMERS = 10

In [ ]:
run_simulation(N_MONs, N_CHAINs, MRs, RRs, avgDP, conv, CTP, PRUNE_OLIGOMERS, "test")

### Run Multiple Simulations

In [2]:
# Number of unique monomers
N_MONs = 4

# Molar ratios of each monomer
# ex. if N_MONs = 4: [0.5, 0.25, 0.20, 0.05]
MRs = [50, 25, 20, 5]

# reactivity ratios of monomers, ex. for 4 monomers:
#[r12, r13, r14]
#[r21, r23, r24]
#[r31, r32, r34]
#[r41, r42, r43]
RRs = ([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]])

# % conversion targetted (0-1), i.e. how much of the monomer pool do you want to use
conv = 0.5

# average degree of polymerization (chain length) you are targetting at YOUR conversion, NOT at 100%.
avgDPs = [50, 100, 150, 200]

# number of polymer chains to simulate
N_CHAINs = 10000

# Chain transfer % (0-1)
# TODO: replace this with direct PDI control (not sure how)
# NOTE TO SELF: for now just fix CTP such that PDI ~ 1.2, as thats roughly the control our RAFT synthesis gives.
CTP = 0.1

# cutoff DP of chains considered as polymers not oligomers that get "purified" out
# set to 0 if you don't want to do any filtration
PRUNE_OLIGOMERS = 10

In [3]:
for avgDP in avgDPs:
    run_simulation(N_MONs, N_CHAINs, MRs, RRs, avgDP, conv, CTP, PRUNE_OLIGOMERS, str(avgDP))

Simulation done! Sequence file saved @ 50.csv
Simulation done! Sequence file saved @ 100.csv
Simulation done! Sequence file saved @ 150.csv
Simulation done! Sequence file saved @ 200.csv
